plot_change.py: Generate significance of electrode weight changes at neuroanatomical sites

## Pseudocode:
    . Import
    . Define functions
    . Input symptom, mode, perdur
    . Load Parameters
    . For Loop through seizures

## Import

In [1]:
import csv
import os
import pathlib
import numpy as np
from scipy.io import loadmat
from scipy.signal import butter,filtfilt

#
import pandas as pd
# from plot_change_module import *

In [2]:
neuroanat_list = ['frontalpole', #FRONTAL LOBE
    'parstriangularis',
    'parsopercularis',
    'parsorbitalis',
    'rostralmiddlefrontal',
    'caudalmiddlefrontal',
    'lateralorbitofrontal',
    'superiorfrontal',
    'medialorbitofrontal',
    'precentral',
    'postcentral', # PARIETAL LOBE
    'inferiorparietal',   
    'superiorparietal',
    'supramarginal',
    'temporalpole', # TEMPORAL LOBE
    'middletemporal',
    'superiortemporal',
    'inferiortemporal',
    'parahippocampal',               
    'Right-Hippocampus',
    'Left-Hippocampus',
    'Right-Amygdala',
    'Left-Amygdala',
    'entorhinal',
    'bankssts',
    'fusiform', # OCCIPITAL LOBE                
    'lingual',
    'Right-Inf-Lat-Vent', # OTHER
    'Right-Cerebral-White-Matter',
    'Left-Cerebral-White-Matter',
    'Right-choroid-plexus',
    'Right-Putamen',
    'Right-VentralDC'];

## Define Functions

In [3]:
def sem_w8s(LL_s,at_onset,before_onset,after_onset):

    row_num = np.shape(LL_s)[0]
    sem_dur = int(after_onset)-int(before_onset)
    
#     print(LL_s[:,int(at_onset):int(after_onset)],1)
#     print('*****************')
#     print(np.mean(LL_s[:,int(at_onset):int(after_onset)],1))
#     # for row in range(0,row_num): #for each channel in LL   
#     LL_meandiff = np.mean(LL_s[:,int(at_onset):int(after_onset)],1) - np.mean(LL_s[:,int(before_onset):int(at_onset)],1)

#     print(LL_meandiff)

In [4]:
def sem_onset(sem_df,sx_input,mx_input,perdur_input,fs):
    
    # match symptom 
    for i in range(0,np.shape(sem_df)[0]): #PUT A LOGICAL VARIABLE -- LOGICAL VECTOR THEN INDEX
        
# mode=2;
# colm=xxxxxx
# at_onset=find(sem_df(:,colm)==mode,1,’first’);
        if sem_df.loc[i,sx_input] == np.float64(mx_input):

            at_onset = np.round((i/5)*fs)
            before_onset = np.round((i/5 - float(perdur_input))*fs)
            after_onset = np.round((i/5 + float(perdur_input))*fs)
            
            return at_onset, before_onset, after_onset 

In [5]:
def ll_sort(LL,anat,neuroanat_list):
    
    a_count = 0
    n_count = 0
    y_count = 0
    
    n_list = []
    y_list = []
    LL_s = np.empty([np.shape(anat)[0],np.shape(LL)[1]])
    LL_s[:] = np.NaN
    
    for n_a in neuroanat_list:
        a_count = 0
        for a_c in anat:
            if np.size(a_c) > 0: 
                if n_a == a_c:
                 #collect ll data
                    n_list.append(n_a) #collect all neuroanat str
                    LL_s[n_count,:] = LL[a_count,:]
                    n_count += 1
            
                    #sort into unique y tick labels
                    if n_a not in y_list:
                        y_list.append(n_a) #collect only non-repeating neuroanat str  
                        # y_count += 1
                        #find index of first and last appearance of neuroanat str, divide by two for ytick
            a_count += 1
    
    return LL_s                     

In [6]:
def ll_transform(llw,fs,d,bl_start,bl_stop):
    
    if bl_start == 0:
        sample_bl_start = 0
    elif bl_start > 0:
        sample_bl_start = (fs*bl_start)[0] #0 because it's python not matlab
    sample_bl_end = (fs*bl_stop)[0]-1
    
    L = int(np.round(llw * fs) - 1) # number of samples to calculate line length
    # LL = np.empty(np.shape(d))
    
    col_len = np.shape(d)[1]-L
    
    LL = np.empty([np.shape(d)[0],col_len])

    LL[:] = np.NaN    
    
    

    
    # Line-length transform
    
    # for row_1 in range(90,100):
    for row_1 in range(0,np.shape(d)[0]-1):
        # LL_list = []
        if np.isnan(d[row_1,0]):
            pass
        else:
            for col_1 in range(sample_bl_start,np.shape(d)[1]-L):
#             for col_1 in range(0,np.shape(d)[1]-L):
                LL[row_1,col_1] = np.sum(np.abs(np.diff(d[row_1,col_1:col_1+L])))
                
            LL_nanmean = np.nanmean(LL[row_1,sample_bl_start:sample_bl_end])
            LL_nanstd = np.nanstd(LL[row_1,sample_bl_start:sample_bl_end])

            LL[row_1,:] = (LL[row_1,:] - LL_nanmean)/LL_nanstd

        
        # print(type(LL[row_1,:]))
        # print(type(LL[row_1,:][0]))
        # print(LL[row_1,:][0])
#         if LL[row_1,:].any() is False:
#             pass
#         else:
#             LL_nanmean = np.nanmean(LL[row_1,sample_bl_start:sample_bl_end])
#             LL_nanstd = np.nanstd(LL[row_1,sample_bl_start:sample_bl_end])

#             LL[row_1,:] = (LL[row_1,:] - LL_nanmean)/LL_nanstd

        # if LL[row_1,:]
            
#         LL_list = LL[row_1,sample_bl_start:np.shape(d)[1]-L].tolist()
        
#         if np.isnan(LL_list).all(): # = 0, get rid of else
#             pass
#         # Normalize LL (to baseline period) as z-scores, "zLL"
#         else:
#              # print('real or zero')
#             LL_nanmean = np.nanmean(LL[row_1,sample_bl_start:sample_bl_end])
#             LL_nanstd = np.nanstd(LL[row_1,sample_bl_start:sample_bl_end])

#             LL[row_1,:] = (LL[row_1,:] - LL_nanmean)/LL_nanstd

    return LL


In [7]:
def open_xl(xl_name): 
    # df = pd.read_excel(xl_name)
    # Ndimensions = df.shape #row, col
    # params_list = df.values.tolist()
    
    df = pd.read_excel(xl_name, index_col=0, engine='openpyxl')

    return df


In [8]:
def filt_perdur(sx_input,mx_input,perdur_input,d,sem_df,fs):
    d_t = d.transpose()
    
    butter_array = np.array([1,(round(fs[0]/2)-1)])
    b,a = butter(2,butter_array/(fs[0]/2),btype='bandpass',output='ba')
    
    mx_index = sem_df.index[sem_df[sx_input]==int(mx_input)][0] # first onset of symptom in ppEEG time 
    onset_mx = round((mx_index/5)*fs[0]) #adjusted for ppEEG time

    ppEEG_perdur = int(perdur_input) * fs[0]

    before_array = d_t[onset_mx - ppEEG_perdur]
    after_array = d_t[onset_mx + ppEEG_perdur]

    before_array = filtfilt(b,a,~np.isnan(before_array),axis=0,padtype='odd',padlen=3*(max(len(b),len(a))-1)).transpose() # d(nns,:) = filtfilt(b,a,d(nns,:)')'; 
    after_array = filtfilt(b,a,~np.isnan(after_array),axis=0,padtype='odd',padlen=3*(max(len(b),len(a))-1)).transpose() # d(nns,:) = filtfilt(b,a,d(nns,:)')'; 

    return before_array, after_array


In [9]:
def load_elecs_anat(pt_path):
                
    #load anatomy and electrode matrix
    os.chdir(pt_path + 'Imaging/elecs')
    e_mat = loadmat('clinical_elecs_all.mat')
    anat_col = e_mat['anatomy'][:,3]
        
    anatomy = np.empty(np.shape(anat_col)).tolist()
    
    for a in range(0,np.shape(anat_col)[0]-1):
        if len(anat_col[a]) > 0:
            anatomy[a] = anat_col[a][0]
    
    return anatomy


In [10]:
def get_params(df_params,pt):
    params_bl_start = df_params.loc[pt]['BLstart']
    params_bl_stop = df_params.loc[pt]['BLstop']
    params_llw = df_params.loc[pt]['llw']

    return params_bl_start, params_bl_stop, params_llw

In [11]:
def get_ptsz(avg_change_path): #Load list of patient and seizure names
    os.chdir(avg_change_path)
    
    df_params = open_xl('OPSCEAparams.xlsx') # Ndimensions and params_list

    with open('sz_name_list.csv','r') as sz_names:
        sz_names_list = list(csv.reader(sz_names))
        sz_rows = np.size(sz_names_list,0) #find row num 
        pt_data = []
        sz_data= []
        for r in range(0,sz_rows):  #collect symptom data for specfic seizure in list
            pt_data.append(sz_names_list[r][0]) 
            sz_data.append(sz_names_list[r][1]) 
    return pt_data, sz_data, df_params

## Input symptom, mode, and duration

In [12]:
sx_input = 'lhx'
mx_input = '2'
perdur_input = '10'

## For loop through patients in list matching seizure

In [13]:
avg_change_path = '/Users/nataliasucher/Desktop/UCSF/Coding/OPSCEA/OPSCEADATA/avg_change_folders/'

pt_data,sz_data,df_params = get_ptsz(avg_change_path) #Ehsan

# for neuro_struc in neuroanat_list:

#For loop through patients
for i in range(len(pt_data)):  #Ehsan
    pt_i = pt_data[i]          #Ehsan
    sz_name = sz_data[i]       #Ehsan

    pt_path = avg_change_path + pt_i + '/'
    os.chdir(pt_path)
    pt_dir = os.listdir(pt_path)

    for sz_name in sz_data: 

        if sz_name in pt_dir: #patients in list matching seizure
 
            print(sz_name)
            
            # PARAMS
            blstart, blstop, llw = get_params(df_params,pt_i) # 2 = params_llw

            # ANAT 
            anat = load_elecs_anat(pt_path)            

            # SEMIOLOGY
            sz_path = pt_path + sz_data[i]
            os.chdir(sz_path)

            sem_df = pd.read_csv(sz_name + '_mat.csv',usecols = [sx_input]) #load semiology matrix .csv             #WANT TO JUST LOAD 1 COLUMN SX_INPUT

            # sx_vec = sem_df.loc[:,sx_input]


            #if FIND FIRST MX_INPUT 
            #if np.where(x==1)[0]
            # time at first mx_input
            # convert fs 
            # if no mx_input is found, then kick out of patient for loop 


#             #DOES SYMPTOM HAVE MX_INPUT DESIRED
#             # def sem_onset(sem_df,sx_input,mx_input,perdur_input,fs):

#                 # match symptom 
#             for i in range(0,np.shape(sem_df)[0]): #PUT A LOGICAL VARIABLE -- LOGICAL VECTOR THEN INDEX

#             # mode=2;
#             # colm=xxxxxx
#             # at_onset=find(sem_df(:,colm)==mode,1,’first’);
#                     if sem_df.loc[i,sx_input] == np.float64(mx_input):

#                         at_onset = np.round((i/5)*fs)
#                         before_onset = np.round((i/5 - float(perdur_input))*fs)
#                         after_onset = np.round((i/5 + float(perdur_input))*fs)

#                         return at_onset, before_onset, after_onset 

            # print(sx_vec)




            # FS & PPEEG
            sz_mat = loadmat(sz_name +'.mat') #load frame speed and ppEEG
            fs = sz_mat['fs'].flatten()
            d = sz_mat['ppEEG'][0:np.shape(anat)[0],(fs*blstart)[0]:] 
            print((fs*blstart)[0])
            print(d)

# #             #remove EKG lines at end of anatomy + ppEEG?
            filt_before, filt_after = filt_perdur(sx_input,mx_input,perdur_input,d,sem_df,fs) #creates 2 arrays of before and after seizure onset and filters
            LL = ll_transform(llw,fs,d,blstart,blstop)
            print(LL)
#             LL_s = ll_sort(LL,anat,neuroanat_list)
#             print(LL_s)
#             at_onset, before_onset, after_onset = sem_onset(sem_df,sx_input,mx_input,perdur_input,fs)

#             sem_w8s(LL_s,at_onset,before_onset,after_onset)

EC91_03
0
[[ 24.14525632  35.22290151  37.50139076 ...   2.4621891    2.60736928
    2.39061944]
 [-23.18900855 -20.56198811 -23.1557212  ...  26.75869829  28.14675899
   25.81868993]
 [-81.99824672 -68.04092836 -55.9516322  ...  46.79421287  44.99489303
   41.11865433]
 ...
 [ 14.10465468   6.90595991   2.24299409 ... 204.3525707  202.35855342
  201.29015667]
 [  7.41092026   8.59101928   7.54449943 ... 113.22951082 111.04977423
  110.44661803]
 [         nan          nan          nan ...          nan          nan
           nan]]
[[ 0.83156162  0.83323153  0.84556761 ... -3.13855766 -3.13869753
  -3.14122328]
 [-0.48101214 -0.47482212 -0.47405987 ... -4.38760794 -4.38525489
  -4.38154001]
 [-0.91427297 -0.96041535 -0.99422308 ... -6.79057683 -6.79910782
  -6.80231278]
 ...
 [-1.52392408 -1.54636496 -1.52119105 ... -5.54371578 -5.53165017
  -5.54094367]
 [-1.57756399 -1.56821699 -1.54385994 ... -4.26729345 -4.2574732
  -4.2557841 ]
 [        nan         nan         nan ...         nan